<a href="https://colab.research.google.com/github/danbernstein/parkingdirty/blob/master/hyperas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pydrive
! pip install hyperas
! pip install hyperopt

from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

# load modules
import os # for accessing project file structure
import requests # for downloading and loading data from the internet
import zipfile # for unzipping zip files 

import numpy.random 
from matplotlib import pyplot # for plotting results
from keras import backend as K


    100% |████████████████████████████████| 993kB 20.9MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive
    100% |████████████████████████████████| 348kB 7.8MB/s 
  Running setup.py bdist_wheel for hyperas ... - done
  Stored in directory: /root/.cache/pip/wheels/06/38/3f/27826f57fae60ef788ceb47e2c649590ab8af31f42075325d2
Successfully built hyperas
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.7 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.15
    Uninstalling prompt-toolkit-1.0.15:
      Successfully uninstalled prompt-toolkit-1.0.15
  [prompt_toolkit]
You must restart the runtime in order to use newly installed versions.


Using TensorFlow backend.


In [0]:
def data():
  
  # download and read in data
  zip_address = 'http://parkingdirty.com/BlockedBikeLaneTrainingFull.zip'
  import numpy as np

  import requests, zipfile, io
  r = requests.get(zip_address)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall() 
  
  # DATA PROCESSING
  
  # append the label at front, and assign to object
  imgs_blocked = list(map('blocked/{0}'.format, os.listdir('blocked')))
  imgs_notblocked = list(map('notblocked/{0}'.format, os.listdir('notblocked')))

  # shuffle data
  numpy.random.shuffle(imgs_blocked)
  numpy.random.shuffle(imgs_notblocked)

  print("number of blocked images: ", len(imgs_blocked))
  print("number of not blocked images: ", len(imgs_notblocked))
  print("ratio of classes: ", len(imgs_blocked)/len(imgs_notblocked))
  
  # separate into training (contains training and validation), and test set
  training_set = imgs_blocked[:int(round(0.8*len(imgs_blocked)))] + imgs_notblocked[:int(round(0.8*len(imgs_notblocked)))]
  print(len(training_set))

  test_set = imgs_blocked[int(round(0.8*len(imgs_blocked))):] + imgs_notblocked[int(round(0.8*len(imgs_notblocked))):]
  print(len(test_set))

  del imgs_blocked
  del imgs_notblocked
  
  import cv2

  training_arrays = [] # images
  training_labels = [] # labels
  test_arrays = []
  test_labels = []
            
            
  for i in training_set:
    img_orig = cv2.imread(i)
    img_arrays = cv2.resize(img_orig, dsize=(150, 150), interpolation=cv2.INTER_CUBIC)
    training_arrays.append(img_arrays)
    
    if i.find("not") is not -1:
      val = 0
      training_labels.append(val)
    else:
      val = 1
      training_labels.append(val)
      
  for i in test_set:
    img_orig = cv2.imread(i)
    img_arrays = cv2.resize(img_orig, dsize=(150, 150), interpolation=cv2.INTER_CUBIC)
    test_arrays.append(img_arrays)
    
    if i.find("not") is not -1:
      val = 0
      test_labels.append(val)
    else:
      val = 1
      test_labels.append(val)
  
  del training_set
  del test_set
  
  X_train_array = np.array(training_arrays)
  Y_train_labels = np.array(training_labels)

  del training_arrays
  del training_labels

  X_test_array = np.array(test_arrays)
  Y_test_labels = np.array(test_labels)

  del test_arrays
  del test_labels
  
  # separate training data into training and validation sets

  from sklearn.model_selection import train_test_split

  X_train, X_val, Y_train, Y_val = train_test_split(X_train_array, Y_train_labels, test_size = 0.2, 
                                                      shuffle = True)
  
  # only resize the image arrays, "resizing" the labels would give 1.0 validation accuracy, which is not true 
  X_train //= 255
  X_val //= 255
  
  print(len(Y_train))
  print(len(Y_val))
  
  train_datagen = ImageDataGenerator(rotation_range=40, # do not need rescale because we did it earlier
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,)


  
  return train_datagen, X_train, Y_train, X_val, Y_val

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title = 'hyperas.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('hyperas.ipynb')

In [4]:
def model_hyp(train_datagen, X_train, Y_train, X_val, Y_val):
    
    batch_size = 32

    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))

    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer= {{choice(['rmsprop', 'adam'])}},
                  metrics=['accuracy'])
    
    model.fit_generator(train_datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        epochs = 1,
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        validation_data=(X_val, Y_val))
        
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    
    print('Val accuracy:', acc)
    print('optimizer:', model.optimizer)
    
    K.clear_session()

    return {'loss': -acc, 'status': STATUS_OK}
  
if __name__ == '__main__':

  best_run, best_model = optim.minimize(model=model_hyp,
                                        data=data,
                                        algo=tpe.suggest,
                                        max_evals=30,
                                        trials=Trials(),
                                        notebook_name='hyperas')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.layers import Conv2D
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    import os
except:
    pass

try:
    import requests
except:
    pass

try:
    import zipfile
except:
    pass

try:
    import numpy.random
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import numpy as np
e

KeyboardInterrupt: ignored